In [1]:
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import calendar
from matplotlib.pyplot import figure
import pymongo
from pymongo import MongoClient
import numpy as np
import math
from datetime import datetime
import statistics
from math import sqrt
from numpy import log as ln
from sklearn.neighbors import KNeighborsRegressor as knr
from sklearn.metrics import confusion_matrix, accuracy_score, r2_score
from sklearn.metrics import roc_auc_score
import time
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression as lr
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('products (1).csv')

In [3]:
df

,Unnamed: 0,_id,Name,Price,Number of Reviews,Category,Score,Sub-Category,SubSub-Category,Reviews,Link,Date,Popularity
0,0,638255f76d55c77b1623ae52,BTS All Member Signature Cotton T-Shirt For Wo...,Rs. 390,378 Ratings,Fashion,4.8/5,Women,Clothing,[[]],https://www.daraz.pk/products/bts-all-member-s...,2022-11-26,0
1,1,63825783a30253d89838a102,Amazing T Shirt Design Summer Collection Shirt...,Rs. 359,26 Ratings,Fashion,4.6/5,Women,Clothing,"[[""There Were Some Stains In The Border Of Thi...",https://www.daraz.pk/products/-i283578056-s152...,2022-11-26,0
2,2,63825784a30253d89838a105,T Shirt for women n girls Summer collection in...,Rs. 389,67 Ratings,Fashion,4.9/5,Women,Clothing,[['wowWwwWww amazing beautiful 😍same as shown ...,https://www.daraz.pk/products/-i277289229-s149...,2022-11-26,0
3,3,63825784a30253d89838a108,Black stylish new hoodi printed tshirt,Rs. 699,No Ratings,Fashion,0/5,Women,Clothing,[[]],https://www.daraz.pk/products/-i362347073-s181...,2022-11-26,0
4,4,63825785a30253d89838a109,Hottest Womens Crystal Rhinestone Bangle Brace...,Rs. 250,6 Ratings,Watches Sunglasses Jewellery,5.0/5,Jewellery,Women,[['I has been risive my parsal bohot acha h I ...,https://www.daraz.pk/products/-i214168547-s186...,2022-11-26,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380559,380559,63b951c3b3bddcfe20adf362,Qiunery 3Pcs/set Girl Kid Swimwear Mermaid Tai...,"Rs. 3,250",3 Ratings,Fashion,5.0/5,Girls,Clothing,[[]],https://www.daraz.pk/products/qiunery-3-3-12-i...,2023-01-06,0
380560,380560,63b951c4b3bddcfe20adf363,Black With Multi Tharri / Wadera Shawl For Him...,"Rs. 1,449",12 Ratings,Fashion,5.0/5,Men,Clothing,[['best product this seller as they are very c...,https://www.daraz.pk/products/shl-112-36-i3747...,2023-01-06,0
380561,380561,63b951c4b3bddcfe20adf364,Kidlove 2pcs Girls Swimsuit Long-sleeved Split...,"Rs. 2,930",2 Ratings,Fashion,5.0/5,Girls,Clothing,[[]],https://www.daraz.pk/products/kidlove-2pcs-gir...,2023-01-06,0
380562,380562,63b951c7b3bddcfe20adf365,Yfashion 2pcs Girls Unicorn Swimsuit Cartoon P...,"Rs. 1,540",1 Ratings,Fashion,5.0/5,Girls,Clothing,[[]],https://www.daraz.pk/products/yfashion-2pcs-gi...,2023-01-06,0


# Preprocessing


In [4]:
df.drop(["_id", 'Unnamed: 0'], axis = 1, inplace = True)

In [5]:
df["Number of Reviews"] = df["Number of Reviews"].str.replace("Ratings","")
df["Number of Reviews"] = df["Number of Reviews"].str.replace("No","0")
df['Number of Reviews'] = df['Number of Reviews'].astype('int')
df["Score"] = df["Score"].str.replace("/5","")
df['Score'] = df['Score'].astype('float')

In [6]:
df

,Name,Price,Number of Reviews,Category,Score,Sub-Category,SubSub-Category,Reviews,Link,Date,Popularity
0,BTS All Member Signature Cotton T-Shirt For Wo...,Rs. 390,378,Fashion,4.8,Women,Clothing,[[]],https://www.daraz.pk/products/bts-all-member-s...,2022-11-26,0
1,Amazing T Shirt Design Summer Collection Shirt...,Rs. 359,26,Fashion,4.6,Women,Clothing,"[[""There Were Some Stains In The Border Of Thi...",https://www.daraz.pk/products/-i283578056-s152...,2022-11-26,0
2,T Shirt for women n girls Summer collection in...,Rs. 389,67,Fashion,4.9,Women,Clothing,[['wowWwwWww amazing beautiful 😍same as shown ...,https://www.daraz.pk/products/-i277289229-s149...,2022-11-26,0
3,Black stylish new hoodi printed tshirt,Rs. 699,0,Fashion,0.0,Women,Clothing,[[]],https://www.daraz.pk/products/-i362347073-s181...,2022-11-26,0
4,Hottest Womens Crystal Rhinestone Bangle Brace...,Rs. 250,6,Watches Sunglasses Jewellery,5.0,Jewellery,Women,[['I has been risive my parsal bohot acha h I ...,https://www.daraz.pk/products/-i214168547-s186...,2022-11-26,0
...,...,...,...,...,...,...,...,...,...,...,...
380559,Qiunery 3Pcs/set Girl Kid Swimwear Mermaid Tai...,"Rs. 3,250",3,Fashion,5.0,Girls,Clothing,[[]],https://www.daraz.pk/products/qiunery-3-3-12-i...,2023-01-06,0
380560,Black With Multi Tharri / Wadera Shawl For Him...,"Rs. 1,449",12,Fashion,5.0,Men,Clothing,[['best product this seller as they are very c...,https://www.daraz.pk/products/shl-112-36-i3747...,2023-01-06,0
380561,Kidlove 2pcs Girls Swimsuit Long-sleeved Split...,"Rs. 2,930",2,Fashion,5.0,Girls,Clothing,[[]],https://www.daraz.pk/products/kidlove-2pcs-gir...,2023-01-06,0
380562,Yfashion 2pcs Girls Unicorn Swimsuit Cartoon P...,"Rs. 1,540",1,Fashion,5.0,Girls,Clothing,[[]],https://www.daraz.pk/products/yfashion-2pcs-gi...,2023-01-06,0


In [7]:
data = pd.DataFrame()
data["Categoric Mapping"] = "Categoric Mapping"
data["Categoric Mapping"] = df["Date"] + "_" + df["Category"] + "/" + df[ "Sub-Category"] + "/" + df["SubSub-Category"]
data["Mean Reviews"] = df["Number of Reviews"]
data["Mean Score"] = df["Score"]
data["Popularity"] = df["Popularity"]

In [8]:
data

,Categoric Mapping,Mean Reviews,Mean Score,Popularity
0,2022-11-26_Fashion/Women/Clothing,378,4.8,0
1,2022-11-26_Fashion/Women/Clothing,26,4.6,0
2,2022-11-26_Fashion/Women/Clothing,67,4.9,0
3,2022-11-26_Fashion/Women/Clothing,0,0.0,0
4,2022-11-26_Watches Sunglasses Jewellery/Jewell...,6,5.0,0
...,...,...,...,...
380559,2023-01-06_Fashion/Girls/Clothing,3,5.0,0
380560,2023-01-06_Fashion/Men/Clothing,12,5.0,0
380561,2023-01-06_Fashion/Girls/Clothing,2,5.0,0
380562,2023-01-06_Fashion/Girls/Clothing,1,5.0,0


In [9]:
value_counts = dict(data["Categoric Mapping"].value_counts())

In [10]:
# data = data.groupby("Categoric Mapping")[["Mean Reviews", "Mean Score" ,"Popularity"]].sum()
# data = pd.DataFrame(data)
# data.reset_index(inplace = True)

In [11]:
data

,Categoric Mapping,Mean Reviews,Mean Score,Popularity
0,2022-11-26_Fashion/Women/Clothing,378,4.8,0
1,2022-11-26_Fashion/Women/Clothing,26,4.6,0
2,2022-11-26_Fashion/Women/Clothing,67,4.9,0
3,2022-11-26_Fashion/Women/Clothing,0,0.0,0
4,2022-11-26_Watches Sunglasses Jewellery/Jewell...,6,5.0,0
...,...,...,...,...
380559,2023-01-06_Fashion/Girls/Clothing,3,5.0,0
380560,2023-01-06_Fashion/Men/Clothing,12,5.0,0
380561,2023-01-06_Fashion/Girls/Clothing,2,5.0,0
380562,2023-01-06_Fashion/Girls/Clothing,1,5.0,0


In [ ]:
keys = list(value_counts.keys())
for i in range(len(data)):
    temp = data["Categoric Mapping"][i]
    try:
        data["Mean Reviews"][i] = data["Mean Reviews"][i] / value_counts[temp]
        data["Mean Score"][i] = data["Mean Score"][i] / value_counts[temp]
        data["Popularity"][i] = data["Popularity"][i] / value_counts[temp]
    except:
        pass

<ipython-input-12-5035b4309e51>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Mean Reviews"][i] = data["Mean Reviews"][i] / value_counts[temp]
<ipython-input-12-5035b4309e51>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Mean Score"][i] = data["Mean Score"][i] / value_counts[temp]
<ipython-input-12-5035b4309e51>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Popularity"][i] = data["Popularity"][i] / value_coun

In [ ]:
data

In [ ]:
dates = []
for i in range(len(data)):
    date = data["Categoric Mapping"][i].split("_")[0]
    dates.append(datetime.strptime(date, "%Y-%m-%d"))
data["Date"] = dates

split_date = datetime.strptime("2022-11-26", "%Y-%m-%d")
split_date
df2 = data.loc[data["Date"] == split_date]
dates = np.unique(dates)

In [ ]:
def divide(df, dates):
    for i in range(len(df)):
        df["Categoric Mapping"][i] = df["Categoric Mapping"][i].split("_")[1]
    
    data_frames = []
    for i in dates:
        data =  (df.loc[df["Date"] == i])
        data_frames.append(data)

    return data_frames

In [ ]:
DATAFRAMES = divide(data, dates)

In [ ]:
data

In [ ]:
#Label = {[(1 + p) × (1 + ΔR)] x μ(S)} / σ(N)
def label_data_set(df):
    df['Rate of Change'] = None
    df['Mean Reviews'] = df['Mean Reviews'] + 1
    for i in range(len(df)):
        cat_name = df['Categoric Mapping'][i]
        lis = df.loc[df['Categoric Mapping'] == cat_name].index.tolist()

        index = lis.index(i)
        if (index == 0):
            df['Rate of Change'][i] = 0
        else:
            index_last = index - 1
            reviews = [int(df['Mean Reviews'][lis[index - 1]]), int(df['Mean Reviews'][lis[index]])]
            reviews_series = pd.Series(reviews)
            df['Rate of Change'][i] = list(reviews_series.pct_change())[1]
    df['Half Label'] = None
    
    for i in range(len(df)):
        value = 1 + float(df['Popularity'][i])
        value2 = 1 + float(df['Rate of Change'][i])
        value = value * value2
        df['Half Label'][i] = value
        
    df['Standard Dev'] = None
    meann = {}
    for i in list(df['Date'].unique()):
        m = df[df['Date'] == i]['Mean Reviews'].mean()
        meann[i] = m
    for i in list(df['Date'].unique()):
        val = statistics.stdev(list(df[df['Date'] == i]['Mean Reviews']))
        df.loc[df['Date'] == i, 'Standard Dev'] = val
        
    df['Label'] = None
    for i in range(len(df)):
        val1 = df['Half Label'][i] * df['Mean Score'][i]
        val1 = val1 / df['Standard Dev'][i]
        df['Label'][i] = val1
  
    df.drop(['Mean Reviews', 'Mean Score', 'Popularity', 'Rate of Change', 'Half Label', 'Standard Dev'], axis = 1, inplace = True)
    return df

In [ ]:
data = label_data_set(data)
data

In [ ]:
data

In [ ]:
df = pd.DataFrame()

df["Category"] = "Category"
df["Sub-Category"] = "Sub-Category"
df["Sub-Sub-Category"] = "Sub-Sub-Category"
df.loc[i, "Year"] = "Year"
df.loc[i, "Month"] = "Month"
df.loc[i, "Day"] = "Day"
df.loc[i, "Date"] = "Date"
df["Label"] = "Label"

for i in range(len(data)):
    df.loc[i, "Category"] = data["Categoric Mapping"][i].split("/")[0]
    df.loc[i, "Sub-Category"] = data["Categoric Mapping"][i].split("/")[1]
    df.loc[i, "Sub-Sub-Category"] = data["Categoric Mapping"][i].split("/")[2]
    df.loc[i, "Year"] = data["Date"][i].year
    df.loc[i, "Month"] = data["Date"][i].month
    df.loc[i, "Day"] = data["Date"][i].day
    df.loc[i, "Date"] = data["Date"][i]
    df.loc[i, "Week_Day"] = data["Date"][i].strftime('%A')

    df.loc[i, "Label"] = data["Label"][i]
df.sort_index(axis = 0)

In [ ]:
df['Categoric Mapping'] = df['Category'] + ' ' + df['Sub-Category'] + ' ' + df['Sub-Sub-Category']

In [95]:
df.to_csv('products.csv', index = False)
